In [3]:
'''To upload to github, all changes you make to a clone of this repo / notebook 
should be stored somewhere and you can just push in a terminal or github desktop'''

'To upload to github, all changes you make to a clone of this repo / notebook should be stored somewhere and you can just push in a terminal or github desktop'

In [ ]:
'''Everything here is made for the bag of words model he showed in Tuesday Nov 19 lecture 
- feel free to change anything, make your own model here or in a different ipynb'''

In [79]:
import gzip
import json


# Path to the .json.gz file
file_path = 'goodreads_reviews_spoiler.json.gz'

# Open and read the file line by line
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    data = []
    for line in f:
        # Parse each line as a separate JSON object
        try:
            review = json.loads(line)
            data.append(review)
        except json.JSONDecodeError as e:
            print(f"Error decoding line: {e}")

# Example: Print the first parsed review
if data:
    print(data[0])
else:
    print("No data was parsed.")


{'user_id': '8842281e1d1347389f2ab93d60773d4d', 'timestamp': '2017-08-30', 'review_sentences': [[0, 'This is a special book.'], [0, 'It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind.'], [0, 'This is what I love about good science fiction - it pushes your thinking about where things can go.'], [0, "It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I've read."], [0, 'For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc.'], [0, 'It is a book about science, and aliens.'], [0, 'The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell.'], [1, 'Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though.'], [1, 'But what woul

In [8]:
from collections import defaultdict
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string
import nltk

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')

# Initialize NLTK tools
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Loop through each entry in the data
for entry in data:
    # Access the 'review_sentences' field
    for _, sentence in entry['review_sentences']:
        # Remove punctuation
        sentence_cleaned = sentence.translate(str.maketrans('', '', string.punctuation))
        # Split into words and convert to lowercase
        words = sentence_cleaned.lower().split()
        # Stem and filter out stopwords
        for word in words:
            if word not in stop_words:
                stemmed_word = stemmer.stem(word)
                total_words += 1
                word_count[stemmed_word] += 1

# Print results
print("Total Words:", total_words)
print("Unique Words:", len(word_count))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tylerlo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total Words: 132997467
Unique Words: 761873


In [18]:
# Build a vector of word counts as (count, word) tuples
word_vector = [(count, word) for word, count in word_count.items()]

# Sort the vector by word count in descending order
word_vector = sorted(word_vector, reverse=True, key=lambda x: x[0])

# Display the top 10 words
print("Top 10 Words by Frequency:")
for count, word in word_vector[:10]:
    print(f"{word}: {count}")


Top 10 Words by Frequency:
book: 3132365
read: 1624647
like: 1580719
love: 1411506
one: 1191420
stori: 1187276
charact: 1135727
realli: 1050887
get: 767997
time: 724121


In [71]:
# Number of top words to extract
nWords = 2000

# Extract the top nWords
top_words = [c[1] for c in word_vector[:nWords]]  # Get the words only
wordId = dict(zip(top_words, range(nWords)))
wordSet = set(top_words)
# Print the first 10 top words for verification
print("Top 10 Words:")
print(top_words[:10])


Top 10 Words:
['book', 'read', 'like', 'love', 'one', 'stori', 'charact', 'realli', 'get', 'time']


False

In [80]:
from sklearn.metrics import mean_squared_error

def feature(datum):
    bow = [0] * len(top_words)  # Initialize bag of words vector
    
    # Combine all sentences in 'review_sentences' into a single text
    sentences = [s[1] for s in datum['review_sentences']]
    r = ' '.join(sentences).lower()  # Combine sentences and convert to lowercase
    
    # Remove punctuation
    r = ''.join(c for c in r if c not in string.punctuation)
    
    # Split into words and count only words in the top_words
    words = r.split()
    for w in words:
        if w in wordSet:  # Only include top words
            bow[wordId[w]] += 1
    
    return bow + [1]  # Add a bias term

# Prepare features (X) and target values (y)
X = [feature(d) for d in data]
y = [d['has_spoiler'] for d in data]  # Use 'has_spoiler' as the target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [81]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

clf = linear_model.Ridge(alpha=1, fit_intercept=False)  # Reduced regularization
clf.fit(X, y)
  # MSE + L2 regularization
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)
weights = list(zip(top_words + ["offset"], theta))


In [86]:

weights[:10]


[('book', -0.0008230817502749303),
 ('read', -0.0033542069754568626),
 ('like', 0.0022223720651001586),
 ('love', -0.0002292461367788715),
 ('one', -0.001464389058951634),
 ('stori', -0.05896772528646231),
 ('charact', -0.09004035890667489),
 ('realli', 0.0),
 ('get', -0.0004563625436604259),
 ('time', -0.0006945155774822217)]